<a href="https://colab.research.google.com/github/SIROHI42/Generative_AI/blob/main/wikipedia_Article_basedLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries
**bold text**

In [ ]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM


# Initialize the Wikipedia loader with a query

In [ ]:
loader = WikipediaLoader(query="indian constitution",load_max_docs=30)
docs = loader.load()

# Initialize a text splitter to break documents into smaller chunks

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = splitter.split_documents(docs)

# Initialize the embedding model

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/tmp/ipython-input-2734244670.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Create a FAISS vector store from the loaded documents using the embedding model

In [ ]:
vectorstore = FAISS.from_documents(docs, embedding_model)

In [ ]:
retriever = vectorstore.as_retriever()

# Load a pre-trained LLM from Hugging Face (Phi-2 model)

In [ ]:
model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


# Define the user query

In [ ]:
query ="indian constitution history??"
print(f"Retrieving chunks for: {query}")

retriever = vectorstore.as_retriever()
relevant_docs = retriever.get_relevant_documents(query)

# Concatenate the content of retrieved documents
context = "\n".join([doc.page_content for doc in relevant_docs])

print("\nRetrieved Context:\n")
print(context)

Retrieving chunks for: indian constitution history??

Retrieved Context:

The Constitution of India is the supreme legal document of India, and the longest written national constitution in the world. The document lays down the framework that demarcates fundamental political code, structure, procedures, powers, and duties of government institutions and sets out fundamental rights, directive principles, and the duties of citizens. 
It espouses constitutional supremacy (not parliamentary supremacy found in the United Kingdom, since it was created by a constituent assembly rather than Parliament) and was adopted with a declaration in its preamble. Although the Indian Constitution does not contain a provision to limit the powers of the parliament to amend the constitution, the Supreme Court in Kesavananda Bharati v. State of Kerala held that there were certain features of the Indian constitution so integral to its functioning and existence that they could never be cut out of the constitutio

Conclusion

In this project, we implemented a retrieval-augmented generative AI pipeline to analyze Wikipedia content. Using a FAISS vector store to semantically index documents about the “Indian Constitution” and a sentence-transformer embedding model to encode text chunks, the system can efficiently retrieve relevant information. The Phi-2 LLM then interprets the retrieved context and generates human-readable answers to queries.

This workflow demonstrates how embedding-based retrieval combined with generative AI enables accurate and context-aware knowledge extraction from large text corpora, providing an effective tool for research, education, and automated information retrieval.